In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../Question-2_Autoencoders")

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler
import torchvision
from torchvision import transforms
import os
from datetime import datetime  # For generating timestamped folder names
from tqdm.auto import tqdm
from CNN_autoencoder_model import *

In [4]:
# Check if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Use the saved model weights

In [5]:
lambda_2_cnn_autoencoder = CNNAutoencoder(2)
lambda_32_cnn_autoencoder = CNNAutoencoder(32)

best_lambda_2_model_path = "../Question-2_Autoencoders/model_ckpts_CNN/2/model_epoch_30.pt"
checkpoint_lambda_2 = torch.load(best_lambda_2_model_path)

best_lambda_32_model_path = "../Question-2_Autoencoders/model_ckpts_CNN/32/model_epoch_30.pt"
checkpoint_lambda_32 = torch.load(best_lambda_32_model_path)

lambda_2_cnn_autoencoder.load_state_dict(checkpoint_lambda_2['model_state_dict'])
lambda_32_cnn_autoencoder.load_state_dict(checkpoint_lambda_32['model_state_dict'])

#move it to GPU
lambda_2_cnn_autoencoder.to(device)
lambda_32_cnn_autoencoder.to(device)

CNNAutoencoder(
  (encoder): CNNEncoder(
    (encoder): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (fc1): Sequential(
      (0): Linear(in_features=392, out_features=392, bias=True)
      (1): ReLU()
    )
    (fc2): Sequential(
      (0): Linear(in_features=392, out_features=32, bias=True)
      (1): ReLU()
    )
  )
  (decoder): CNNDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=32, out_features=392, bias=True)
      (1): ReLU()
      (2): Unflatten(dim=1, unflattened_size=(8, 7, 7))
      (3): ConvTranspose2d(8, 8, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (4): ReLU()
      (5): ConvTranspose2d(8, 16, kernel_size=(3, 

# Load the mnist data

In [6]:
# Load MNIST dataset and apply transformations
transform = transforms.Compose([transforms.ToTensor()])
mnist_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(mnist_dataset, batch_size=1, shuffle=False)

# Now pass the whole dataset through the encoder and get the latent embeddings

### First lets see lambda = 2

In [12]:
#set in eval mode
lambda_2_cnn_autoencoder.encoder.eval()
lambda_2_cnn_autoencoder.decoder.eval()

lambda_2_all_latent_features = []

with torch.no_grad():
    for batch_idx, data in tqdm(enumerate(train_loader)):
        images, _ = data
        images = images.to(device)
        # Forward pass through the model
        latent_features_batch = lambda_2_cnn_autoencoder.encoder(images)
        lambda_2_all_latent_features.append(latent_features_batch)


0it [00:00, ?it/s]

In [18]:
all_latent_features[0].size()

torch.Size([1, 2])